In [ ]:
!pip install spacy
!pip install -qU pypdf2
!pip install langchain
!pip install langchain_community
!pip install pypdf
!pip install spacy[transformers]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.24
    Uninstalling langchain-core-0.3.24:
      Successfully uninstalled langchain-core-0.3.24
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.2
    Uninstalling langchain-text-splitters-0.3.2:
      Successfully uninstalled langchain-text-splitters-0.3.2
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.11
    Uninstalling langchain-0.3.11:
      Successfully uninstalled langchain-0.3.11
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
from langchain_community.document_loaders import PyPDFLoader
from spacy.tokens import DocBin
import json
import numpy as np
import os
import pandas as pd
import random
import re
import spacy
import spacy.util
import torch
drive.mount("/content/drive/") # load dataset drive

Mounted at /content/drive/


In [ ]:
base_path = "/content/drive/MyDrive/Tubes_NLP"
dataset_path = "/content/drive/MyDrive/Tubes_NLP/Datasets"
test_dataset_path = "/content/drive/MyDrive/Tubes_NLP/Test Datasets"

In [ ]:

import spacy
from spacy.tokens import DocBin

json_files = [f"{base_path}/json/acetone.json",
              f"{base_path}/json/ammonium-chloride.json",
              f"{base_path}/json/benzyl-alcohol.json",
              f"{base_path}/json/caffeine.json",
              f"{base_path}/json/dichloromethane.json",
              f"{base_path}/json/ethanol.json",
              f"{base_path}/json/heptane.json",
              f"{base_path}/json/hexane.json",
              f"{base_path}/json/mct-oil-regular.json",
              f"{base_path}/json/methanol.json",
              f"{base_path}/json/sodium-chloride.json",
              f"{base_path}/json/tetrahydrofuran.json",
              f"{base_path}/json/triethanolamine.json",
              f"{base_path}/json/water.json",
              f"{base_path}/json/xylenes.json",]

doc_bin = DocBin()
nlp = spacy.blank("en")

for file in json_files:
    with open(file, "r") as f:
        data = json.load(f)
        annotations = data["annotations"]
        for annotation in annotations:
            text = annotation[0]
            entities = annotation[1]
            doc = nlp.make_doc(text)
            ents = []
            for start, end, label in entities["entities"]:
                span = doc.char_span(start, end, label=label)
                if span:
                    ents.append(span)
            doc.ents = ents
            doc_bin.add(doc)

doc_bin.to_disk("train.spacy")

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [ ]:
!python -m spacy init config config_bert.cfg --lang en --pipeline transformer,ner --optimize efficiency


✘ The provided output file already exists. To force overwriting the
config file, set the --force or -F flag.



In [ ]:
!python -m spacy train config_bert.cfg --paths.train ./train.spacy --paths.dev ./train.spacy --output ./output --gpu-id 0

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    854.78

In [ ]:
import PyPDF2

pdf_path = f"{test_dataset_path}/getpdf.pdf"

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

pdf_text = extract_text_from_pdf(pdf_path)

In [ ]:
import spacy

nlp = spacy.load("./output/model-last")
doc = nlp(pdf_text)

for ent in doc.ents:
    print(f"Entity: {ent.text}, Label: {ent.label_}")

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Entity: Darmstadt, Label: CHEMICAL_NAME
Entity: Calcium Hydroxide, Label: CHEMICAL_NAME
Entity: 1305-62-0, Label: CAS_NUMBER
Entity: Darmstadt, Label: CHEMICAL_NAME
Entity: soap, Label: CHEMICAL_NAME
Entity: water., Label: CHEMICAL_NAME
Entity: Ca(OH)2, Label: CHEMICAL_NAME
Entity: 1305-62-0, Label: CAS_NUMBER
Entity: Calcium hydroxide, Label: CHEMICAL_NAME
Entity: Darmstadt, Label: CHEMICAL_NAME
Entity: water., Label: CHEMICAL_NAME
Entity: water, Label: CHEMICAL_NAME
Entity: Calcium oxide, Label: CHEMICAL_NAME
Entity: water, Label: CHEMICAL_NAME
Entity: water, Label: CHEMICAL_NAME
Entity: water, Label: CHEMICAL_NAME
Entity: Darmstadt, Label: CHEMICAL_NAME
Entity: Calcium hydroxide, Label: CHEMICAL_NAME
Entity: 1305-62-0, Label: CAS_NUMBER
Entity: Darmstadt, Label: CHEMICAL_NAME
Entity: www.kcl.de, Label: CHEMICAL_NAME
Entity: Dermatril, Label: CHEMICAL_NAME
Entity: www.kcl.de, Label: CHEMICAL_NAME
Entity: Dermatril, Label: CHEMICAL_NAME
Entity: b, Label: CHEMICAL_NAME
Entity: c, Label

In [ ]:
import json
import spacy
from spacy.tokens import DocBin

json_files = [f"{base_path}/json/testpdf (1).json",
              f"{base_path}/json/testpdf (2).json",
              f"{base_path}/json/testpdf (3).json",
              f"{base_path}/json/testpdf (4).json",
              f"{base_path}/json/testpdf (5).json",
              f"{base_path}/json/testpdf (6).json",
              f"{base_path}/json/testpdf (7).json",
              f"{base_path}/json/testpdf (8).json",
              f"{base_path}/json/testpdf (9).json",
              f"{base_path}/json/testpdf (10).json"]

doc_bin = DocBin()
nlp = spacy.blank("en")

for file in json_files:
    with open(file, "r") as f:
        data = json.load(f)
        annotations = data["annotations"]
        for annotation in annotations:
            text = annotation[0]
            entities = annotation[1]
            doc = nlp.make_doc(text)
            ents = []
            for start, end, label in entities["entities"]:
                span = doc.char_span(start, end, label=label)
                if span:
                    ents.append(span)
            doc.ents = ents
            doc_bin.add(doc)

doc_bin.to_disk("test.spacy")

In [ ]:
!python -m spacy evaluate ./output/model-last ./test.spacy --gpu-id 0

ℹ Using GPU: 0
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(

================================== Results ==================================

TOK     100.00
NER P   58.87 
NER R   62.57 
NER F   60.67 
SPEED   15073 


=============================== NER (per type) ===============================

                    P       R       F
CHEMICAL_NAME   61.71   64.84   63.24
CAS_NUMBER      48.15   97.50   64.46
COMPANY          0.00    0.00    0.00
FORMULA         80.00   36.36   50.00

